In [7]:
import pandas as pd

# Cargar el archivo de datos geográficos limpios
geo_data = pd.read_parquet('Data/datos_geograficos_limpios.parquet')

# Asegurarse de que los datos contienen las columnas necesarias
required_columns = ['latitude', 'longitude', 'elevation', 'timezone']
if not all(col in geo_data.columns for col in required_columns):
    raise ValueError(f"El archivo debe contener las columnas: {', '.join(required_columns)}")

# Crear el geo_join_id como identificador único
geo_data['geo_join_id'] = range(1, len(geo_data) + 1)

Vehicles

In [ ]:
import pandas as pd

# Cargar el archivo CSV con los datos de costos operacionales de vehículos
costo_operacional_vehiculos = pd.read_csv('Data/costo_operacional_vehiculos_clean.csv')

# Crear el dataset Vehicles a partir del archivo CSV
vehicles_dataset = costo_operacional_vehiculos[['Manuf', 'Model', 'Fuel_Type', 'Fuel_Cost', 'Electric_Cost', 'Total_Cost', 'Noise_Level']].copy()

# Renombrar las columnas para que coincidan con la estructura deseada
vehicles_dataset.rename(columns={
    'Manuf': 'manuf',
    'Model': 'model',
    'Fuel_Type': 'fuel_type',
    'Fuel_Cost': 'fuel_cost',
    'Electric_Cost': 'electric_cost',
    'Total_Cost': 'total_cost',
    'Noise_Level': 'noise_level'
}, inplace=True)

# El campo 'model' será la clave primaria (PK)
# Asegúrate de que 'model' sea único
vehicles_dataset = vehicles_dataset.drop_duplicates(subset='model')

# Guardar el nuevo dataset
vehicles_dataset.to_parquet('etl_minio/vehicles.parquet', index=False)

print("El dataset 'Vehicles' ha sido creado y guardado en 'vehicles.parquet'.")


Trips (Yellow, Green, FHV)

In [ ]:
import pandas as pd

# Cargar los datos
yellow_tripdata = pd.read_parquet('Data/yellow_tripdata.parquet')
green_tripdata = pd.read_parquet('Data/green_tripdata.parquet')
fhv_tripdata = pd.read_parquet('Data/fhv_tripdata.parquet')
geographic_data = pd.read_parquet('Data/datos_geograficos_limpios.parquet')
meteorological_data = pd.read_parquet('Data/datos_meteorologicos_limpios.parquet')

# Agregar la columna 'tipo' a cada DataFrame
yellow_tripdata['tipo'] = 'Yellow'
green_tripdata['tipo'] = 'Green'
fhv_tripdata['tipo'] = 'FHV'

# Crear DataFrame combinado
combined_tripdata = pd.concat([yellow_tripdata, green_tripdata, fhv_tripdata], ignore_index=True)

# Crear un DataFrame para las ubicaciones geográficas
geo_data = geographic_data[['latitude', 'longitude', 'geo_join_id']].drop_duplicates().reset_index(drop=True)

# Unir el DataFrame combinado con datos geográficos por PULocationID y DOLocationID
combined_tripdata = combined_tripdata.merge(geo_data, left_on='PULocationID', right_on='geo_join_id', suffixes=('', '_pickup'))
combined_tripdata = combined_tripdata.merge(geo_data, left_on='DOLocationID', right_on='geo_join_id', suffixes=('', '_dropoff'))

# Seleccionar y renombrar las columnas para el nuevo dataset
combined_tripdata_dataset = combined_tripdata[['VendorID', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'total_amount', 'PULocationID', 'DOLocationID', 'payment_type', 'congestion_surcharge', 'tipo']].copy()

# Limpiar las columnas para la clave foránea
combined_tripdata_dataset.rename(columns={
    'VendorID': 'vendor_id',
    'pickup_datetime': 'pickup_datetime',
    'dropoff_datetime': 'dropoff_datetime',
    'passenger_count': 'passenger_count',
    'trip_distance': 'trip_distance',
    'total_amount': 'total_amount',
    'PULocationID': 'PULocationID',
    'DOLocationID': 'DOLocationID',
    'payment_type': 'payment_type',
    'congestion_surcharge': 'congestion_surcharge',
    'tipo': 'tipo'
}, inplace=True)

# Unir con datos meteorológicos por la fecha y hora de recogida
combined_tripdata_dataset = combined_tripdata_dataset.merge(meteorological_data[['Time', 'Temperature_2m (°C)', 'rain (mm)', 'is_day']], left_on='pickup_datetime', right_on='Time', how='left')

# Seleccionar las columnas necesarias
combined_tripdata_dataset = combined_tripdata_dataset[['vendor_id', 'pickup_datetime', 'dropoff_datetime', 'passenger_count', 'trip_distance', 'total_amount', 'PULocationID', 'DOLocationID', 'payment_type', 'congestion_surcharge', 'tipo', 'Temperature_2m (°C)', 'rain (mm)', 'is_day']]

# Asegurarse de que los datos estén completos y eliminar duplicados
combined_tripdata_dataset = combined_tripdata_dataset.drop_duplicates()

# Guardar el nuevo dataset
combined_tripdata_dataset.to_parquet('etl_minio/trips.parquet', index=False)

print("El dataset combinado de viajes ha sido creado y guardado en 'trips.parquet'.")


Geographic Data

In [ ]:
import pandas as pd

# Cargar los datos geográficos desde el archivo parquet
geographic_data = pd.read_parquet('Data/datos_geograficos_limpios.parquet')

# Seleccionar y renombrar las columnas para el nuevo dataset
geographic_data_dataset = geographic_data[['geo_join_id', 'latitude', 'longitude', 'elevation', 'timezone']].copy()

# Asegurarse de que 'geo_join_id' sea único (clave primaria)
geographic_data_dataset.drop_duplicates(subset=['geo_join_id'], inplace=True)

# Guardar el nuevo dataset
geographic_data_dataset.to_parquet('etl_minio/geographic_data.parquet', index=False)

print("El dataset de datos geográficos ha sido creado y guardado en 'geographic_data.parquet'.")


 Air Quality Measurement

In [ ]:
import pandas as pd

# Cargar el archivo parquet con datos de calidad del aire
air_quality = pd.read_parquet('../air_quality_cleaned.parquet')

# Seleccionar y renombrar las columnas para el nuevo dataset
air_quality_dataset = air_quality[['Nombre del Indicador', 'Medida del indicador', 'Geo Place Name', 'Time Period', 'Data Value', 'Year']].copy()

# Renombrar las columnas para que coincidan con la estructura proporcionada
air_quality_dataset.rename(columns={
    'Nombre del Indicador': 'name',
    'Medida del indicador': 'measure',
    'Geo Place Name': 'geo_place_name',
    'Time Period': 'time_period',
    'Data Value': 'data_value',
    'Year': 'year'
}, inplace=True)

# Cargar el dataset de datos geográficos para obtener geo_join_id
geographic_data = pd.read_parquet('../datos_geograficos_limpios.parquet')

# Asegurarse de que geo_place_name esté presente en ambos datasets para crear geo_join_id
# Hacer una fusión para obtener geo_join_id
merged_data = pd.merge(air_quality_dataset, geographic_data[['geo_place_name', 'geo_join_id']], 
                       on='geo_place_name', how='left')

# Seleccionar y renombrar las columnas para el dataset final
final_air_quality_dataset = merged_data[['indicator_id', 'name', 'measure', 'geo_join_id', 'data_value', 'time_period', 'year']].copy()

# Asignar un identificador único para cada indicador (esto puede ser generado o asignado en otro proceso)
final_air_quality_dataset['indicator_id'] = final_air_quality_dataset.index + 1

# Asegurarse de que 'geo_join_id' esté presente y no tenga valores nulos
final_air_quality_dataset = final_air_quality_dataset.dropna(subset=['geo_join_id'])

# Guardar el nuevo dataset
final_air_quality_dataset.to_parquet('../etl_minio/air_quality_measurement.parquet', index=False)

print("El dataset de calidad del aire ha sido creado y guardado en 'air_quality_measurement.parquet'.")



 Meteorological Data

In [ ]:
import pandas as pd

# Cargar los datos meteorológicos desde el archivo parquet
meteorological_data = pd.read_parquet('../datos_meteorologicos_limpios.parquet')

# Seleccionar y renombrar las columnas para el nuevo dataset
meteorological_data_dataset = meteorological_data[['Time', 'Temperature_2m (°C)', 'rain (mm)', 'is_day']].copy()

# Renombrar las columnas para que coincidan con la estructura proporcionada
meteorological_data_dataset.rename(columns={
    'Time': 'time',
    'Temperature_2m (°C)': 'temperature_2m',
    'rain (mm)': 'rain',
    'is_day': 'is_day'
}, inplace=True)

# Asegurarse de que 'time' sea único (clave primaria)
meteorological_data_dataset.drop_duplicates(subset=['time'], inplace=True)

# Guardar el nuevo dataset
meteorological_data_dataset.to_parquet('../etl_minio/meteorological_data.parquet', index=False)

print("El dataset de datos meteorológicos ha sido creado y guardado en 'meteorological_data.parquet'.")


Fuel Stations

In [ ]:
import pandas as pd

# Cargar los datos de estaciones de combustible desde el archivo parquet
fuel_stations = pd.read_parquet('../df_eafcs.parquet')

# Seleccionar y renombrar las columnas para el nuevo dataset
fuel_stations_dataset = fuel_stations[['Station Name', 'Street Address', 'City', 'State', 'Fuel Type Code']].copy()

# Renombrar las columnas para que coincidan con la estructura proporcionada
fuel_stations_dataset.rename(columns={
    'Station Name': 'station_name',
    'Street Address': 'street_address',
    'City': 'city',
    'State': 'state',
    'Fuel Type Code': 'fuel_type_code'
}, inplace=True)

# Asegurarse de que 'fuel_type_code' sea único (clave primaria)
fuel_stations_dataset.drop_duplicates(subset=['fuel_type_code'], inplace=True)

# Guardar el nuevo dataset
fuel_stations_dataset.to_parquet('../etl_minio/fuel_stations.parquet', index=False)

print("El dataset de estaciones de combustible ha sido creado y guardado en 'fuel_stations.parquet'.")


 Fuel Economy Data

In [ ]:
import pandas as pd

# Cargar los datos de economía de combustible desde el archivo parquet
fuel_economy_data = pd.read_parquet('../df_vfed.parquet')

# Seleccionar y renombrar las columnas para el nuevo dataset
fuel_economy_data_dataset = fuel_economy_data[['Manufacturer', 'Model', 'Miles per gallon (mpg)', 'CO2 (g p/mile)', 'FuelCost']].copy()

# Renombrar las columnas para que coincidan con la estructura proporcionada
fuel_economy_data_dataset.rename(columns={
    'Manufacturer': 'manufacturer',
    'Model': 'model',
    'Miles per gallon (mpg)': 'miles_per_gallon',
    'CO2 (g p/mile)': 'co2_per_mile',
    'FuelCost': 'fuel_cost'
}, inplace=True)

# Asegurarse de que 'model' sea único (clave primaria)
fuel_economy_data_dataset.drop_duplicates(subset=['model'], inplace=True)

# Guardar el nuevo dataset
fuel_economy_data_dataset.to_csv('../etl_minio/fuel_economy_data.parquet', index=False)

print("El dataset de economía de combustible ha sido creado y guardado en 'fuel_economy_data.parquet'.")
